# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [3]:
import pandas as pd
import numpy as np
from scipy import stats

In [4]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [5]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [6]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


### What test is appropriate for this problem? Does CLT apply?

The sample size is greater than 20 for each divided set as well as for the whole. Each entry is also independent since 
no resume has an influence on any other resumes here. Therefore the CLT applies. 

I will use bootstrap and frequentist approaches to test the difference of means between the amount of 
calls resumes with black or white sounding names received.

### What are the null and alternate hypotheses?

H0: The true difference of means between blacks and whites for resume call backs is zero.
Ha: The true difference means between blacks and whites for resume call backs is not zero.
    
In simpler terms, the null hypothesis is that the sound of the name has no effect on the calls column,
and the alternative hypothesis states the opposite.



In [ ]:
#Functions for bootstrap hypothesis test
def bootstrap_replicate_1d(data, func):
    return func(np.random.choice(data, size=len(data)))

def draw_bs_reps(data, func, size=1):
    """Draw bootstrap replicates."""

    # Initialize array of replicates: bs_replicates
    bs_replicates = np.empty(size)

    # Generate replicates
    for i in range(size):
        bs_replicates[i] = bootstrap_replicate_1d(data, func)

    return bs_replicates

In [9]:
#Divide the data
w = data[data.race=='w']
b = data[data.race=='b']

In [10]:
#Select relevant columns
w_calls = w.call
b_calls = b.call

In [11]:
"""
Bootstrap Approach: hypothesis test for difference of means, confidence interval, and margin of error
"""
#Hypothesis Test
mean_force = np.mean(data.call)
empirical_diff_means = np.mean(w_calls) - np.mean(b_calls)
# Generate shifted arrays
force_w_shifted = w_calls - np.mean(w_calls) + mean_force
force_b_shifted = b_calls - np.mean(b_calls) + mean_force

# Compute 10,000 bootstrap replicates from shifted arrays
bs_replicates_w = draw_bs_reps(force_w_shifted, np.mean, size = 10000)
bs_replicates_b = draw_bs_reps(force_b_shifted, np.mean, size = 10000)

# Get replicates of difference of means: bs_replicates
bs_replicates =  bs_replicates_w - bs_replicates_b

# Compute and print p-value: p
p = np.sum(bs_replicates >= empirical_diff_means)/len(data)
print('p-value =', p)

p-value = 0.0


In [21]:
#Confidence interval for the difference of means
conf_int = np.percentile(bs_replicates, [2.5, 97.5])
print('95% confidence interval =', conf_int)

95% confidence interval = [-0.01478443  0.01519505]


In [19]:
#Find Margin of error for the confidence interval of the difference of means.
p = 0.95
df = len(bs_replicates) - 1
sem = np.std(bs_replicates) / np.sqrt(len(bs_replicates))
value = stats.t.ppf(p, df)
me = value * sem
print('Margin of Error', np.abs(me)*100, '%')

Margin of Error 0.012772236866453278 %


In [23]:
"""
Frequentist Approach: two sample t-test, confidence interval, and margin of error
"""
#Hypothesis test
stat, p = stats.ttest_ind(w_calls, b_calls)
print("p = ", p)

p =  3.940802103128886e-05


In [47]:
#Confidence interval
empirical_diff = w_calls.subtract(b_calls, fill_value=0)
conf_int = np.percentile(empirical_diff, [2.5, 97.5])
print('95% confidence interval =', conf_int)

95% confidence interval = [-1.  1.]


In [44]:
#Find Margin of error for the confidence interval of the difference of means.
p = 0.95
df = len(empirical_diff) - 1
sem = np.std(empirical_diff) / np.sqrt(len(empirical_diff))
value = stats.t.ppf(p, df)
me = value * sem
print('Margin of Error', np.abs(me)*100, '%')

Margin of Error 0.6677641609320806 %


### Conclusion:

Both hypothesis tests rejected the null hypothesis. This means that the true difference of means was not equal to zero.

In other words, the sound of the name, whether black or white, did likely have an effect on the number of call backs
applicants received. On the other hand, the confidence interval for the bootstrap approach was arbitrarily close to zero with a small
margin of error. The two sample t-test also included zero it its confidence interval with a relatively small margin of error.

This contradicts the hypothesis test results. Since the names were assigned randomly, it is hard to check for lurking variables in the applicants themselves.

It is likely that race is not the most important factor for callbacks while it does seem to have influence. Other factors such as education and years of experience likely play a large role in the selection process too.

I would add these factors into my analysis by bucketing groups further. Checking for resumes with similar amounts of experience, and then checking the effect of the names.
